In [1]:
"""
Script de Pré-processamento de Dados para Análise de Sentimentos

Este script é responsável pelo pré-processamento de um dataset de textos para serem usados na análise de sentimentos. 
O objetivo é limpar e preparar os dados para que modelos de machine learning possam operar de forma eficiente. 
Este processo inclui a remoção de URLs, menções de usuários, hashtags, caracteres especiais, e palavras de parada (stopwords). 
Adicionalmente, o script realiza o balanceamento de rótulos para evitar viéses em categorias com super-representação.

Funcionalidades:
- Limpeza de textos para remover elementos indesejáveis como URLs, menções, hashtags e caracteres não alfabéticos.
- Remoção de stopwords para reduzir a dimensionalidade dos dados e focar em palavras significativas.
- Balanceamento de rótulos através de undersampling, garantindo que todas as classes de sentimentos sejam igualmente representadas.
- Filtragem por idioma para processar apenas textos em inglês.

O script é modular, permitindo que cada função seja reutilizada em outros contextos de pré-processamento de texto conforme necessário.

Dependências:
- Pandas: Para manipulação de dados.
- NLTK: Para ferramentas de processamento de texto como tokenização e remoção de stopwords.
- Scikit-learn: Para técnicas de resampling usadas no balanceamento de rótulos.
- Regex (re): Para expressões regulares utilizadas na limpeza de texto.

O fluxo do script inclui carregar os dados, aplicar funções de limpeza e remoção de stopwords, balancear os rótulos e salvar o dataset processado para uso posterior. 

Membros do grupo:

Alonso Batista de Oliveira Júnior
André Moreira de Carvalho
Gustavo Castro Candeia
Halex Maciel Silva Vieira
Welbert Luiz Silva Junior

"""


import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.utils import resample
from typing import List, Tuple

In [2]:
# Fazendo o download de recursos necessários do NLTK para processamento de texto
nltk.download('stopwords')
nltk.download('punkt')

# Definindo as stopwords em inglês como uma constante para uso repetido, o que melhora a eficiência
STOP_WORDS_EN = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\halex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def clean_url_mentions(text: str) -> str:
    """Remove URLs e menções de usuários do texto.
    
    Args:
        text (str): Texto original.
        
    Returns:
        str: Texto limpo sem URLs e menções.
    """
    text = re.sub(r'http\S+', '', text)  # Remove URLs que começam com http
    text = re.sub(r"www.\S+", '', text)  # Remove URLs que começam com www
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove menções a usuários
    return text


In [4]:
def clean_hashtags_special_chars(text: str) -> str:
    """Remove hashtags e caracteres especiais do texto.
    
    Args:
        text (str): Texto a ser limpo.
        
    Returns:
        str: Texto limpo de hashtags e caracteres não alfabéticos.
    """
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)  # Remove hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove caracteres que não são letras ou espaços
    return text

In [5]:
def clean_text(text: str) -> str:
    """Orquestra a limpeza de URLs, menções, hashtags e caracteres especiais no texto.
    
    Args:
        text (str): Texto original possivelmente contendo URLs, menções, hashtags e caracteres especiais.
    
    Returns:
        str: Texto limpo.
    """
    if pd.isnull(text):
        return ""
    text = clean_url_mentions(text)
    text = clean_hashtags_special_chars(text)
    return re.sub(r'\s+', ' ', text).strip().lower()  # Reduz múltiplos espaços para um único espaço

In [6]:
def remove_stopwords(text: str) -> str:
    """Remove palavras de parada (stopwords) do texto.
    
    Args:
        text (str): Texto original tokenizado.
    
    Returns:
        str: Texto sem stopwords.
    """
    tokens = word_tokenize(text)  # Tokeniza o texto em palavras individuais
    return ' '.join([token.lower() for token in tokens if token.lower() not in STOP_WORDS_EN])

In [7]:
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica as funções de pré-processamento no dataframe e filtra dados relevantes.
    
    Args:
        df (pd.DataFrame): Dataframe contendo coluna de texto para limpeza.
    
    Returns:
        pd.DataFrame: Dataframe filtrado e processado.
    """
    df['Clean_Text'] = df['Text'].apply(clean_text)  # Limpeza inicial do texto
    df['Clean_Text_LSTM'] = df['Text'].apply(clean_text) # Limpeza inicial do texto para uso em LSTM (Mantém as stop words) 
    df_filtered = df[(df['Language'] == 'en')].copy()  # Filtra por idioma inglês 
    df_filtered['Clean_Text'] = df_filtered['Clean_Text'].apply(remove_stopwords)  # Remove stopwords
    df_filtered = df_filtered.drop_duplicates(subset=['Clean_Text'])  # Remove textos duplicados
    return df_filtered

In [8]:
def balance_labels(df: pd.DataFrame, labels: List[str]) -> pd.DataFrame:
    """Realiza undersampling para balancear os rótulos.
    
    Args:
        df (pd.DataFrame): Dataframe com desbalanceamento de rótulos.
        labels (List[str]): Lista de rótulos a serem balanceados.
    
    Returns:
        pd.DataFrame: Dataframe com rótulos balanceados.
    """
    label_counts = df['Label'].value_counts()
    min_count = label_counts.min()
    dfs = []
    for label in labels:
        df_label = df[df['Label'] == label]
        df_label_downsampled = resample(df_label, replace=False, n_samples=min_count, random_state=42)
        dfs.append(df_label_downsampled)
    return pd.concat(dfs).sample(frac=1).reset_index(drop=True)

In [9]:
# Configuração de caminhos de arquivo
data_dir = '../data'
input_file = os.path.join(data_dir, 'dataset.csv')
output_file = os.path.join(data_dir, 'cleaned_dataset.csv')

In [10]:
# Carrega dados
df = pd.read_csv(input_file)
print(df['Label'].value_counts())  # Mostra a distribuição inicial dos rótulos

Label
positive       264545
negative       262220
uncertainty    206940
litigious      204149
Name: count, dtype: int64


In [11]:
# Processa dados
df_processed = preprocess_dataframe(df)
print(df_processed['Label'].value_counts())  # Mostra a distribuição dos rótulos após processamento
df_processed.head()  # Exibe as primeiras linhas do dataframe processado

Label
negative       229658
positive       221604
uncertainty    188989
litigious      164189
Name: count, dtype: int64


,Text,Language,Label,Clean_Text,Clean_Text_LSTM
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious,testimony evidence court law state federal mus...,testimony is not evidence in a court of law st...
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious,flagstar bank discloses data breach impacted m...,flagstar bank discloses a data breach that imp...
3,Rwanda is set to host the headquarters of Unit...,en,positive,rwanda set host headquarters united nations de...,rwanda is set to host the headquarters of unit...
4,OOPS. I typed her name incorrectly (today’s br...,en,litigious,oops typed name incorrectly todays brave witne...,oops i typed her name incorrectly todays brave...
5,It sucks for me since I'm focused on the natur...,en,negative,sucks since im focused nature aspect things en...,it sucks for me since im focused on the nature...


In [12]:
# Balanço de Rótulos
labels = ['positive', 'negative', 'uncertainty', 'litigious']
df_balanced = balance_labels(df_processed, labels)
print(df_balanced['Label'].value_counts())  # Mostra a distribuição dos rótulos após balanceamento

Label
uncertainty    164189
litigious      164189
negative       164189
positive       164189
Name: count, dtype: int64


In [13]:
df_balanced.columns

# seleciona apenas as colunas necessárias
df_balanced = df_balanced[['Clean_Text', 'Clean_Text_LSTM','Label']]
df_balanced.head()

,Clean_Text,Clean_Text_LSTM,Label
0,clear lebron ready post anything social media ...,it was clear lebron was not ready to post anyt...,uncertainty
1,maybe stop tweeting hire good criminal defense...,maybe you should stop tweeting and hire some g...,litigious
2,anyway main question four domains match lovesi...,anyway the main question is which of these fou...,negative
3,ah ok thanks saw clips looked like quite heavy...,ah ok thanks for that just saw some clips of w...,uncertainty
4,hi ravi ji seen trades looks solid wanted conn...,hi ravi ji have seen few of your trades and lo...,uncertainty


In [14]:
# Salva dados processados e balanceados
df_balanced.to_csv(output_file, index=False)